# This is the labelling and preparation of our own data which we collected from Twitter (X)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from openai import OpenAI
import os
import re
from openai import OpenAI
import time
import random
import re

/kaggle/input/vevfvs/Raw X Data/inference_tweets2_10_04.csv
/kaggle/input/vevfvs/Raw X Data/amazing_tweets_05_04.csv
/kaggle/input/vevfvs/Raw X Data/inference_tweets1_10_04.csv
/kaggle/input/vevfvs/Raw X Data/horrified11_04.csv
/kaggle/input/vevfvs/Raw X Data/inference_tweets_05_04.csv
/kaggle/input/vevfvs/Raw X Data/worse_tweets_05_04.csv
/kaggle/input/vevfvs/Raw X Data/furious_tweets_11_04.csv
/kaggle/input/vevfvs/Raw X Data/inference_tweets_05_04_1.csv


In [ ]:
#Combine all files to one

import glob

# List of file paths
file_paths = [
    '/kaggle/input/vevfvs/Raw X Data/inference_tweets2_10_04.csv',
    '/kaggle/input/vevfvs/Raw X Data/amazing_tweets_05_04.csv',
    '/kaggle/input/vevfvs/Raw X Data/inference_tweets1_10_04.csv',
    '/kaggle/input/vevfvs/Raw X Data/horrified11_04.csv',
    '/kaggle/input/vevfvs/Raw X Data/inference_tweets_05_04.csv',
    '/kaggle/input/vevfvs/Raw X Data/worse_tweets_05_04.csv',
    '/kaggle/input/vevfvs/Raw X Data/furious_tweets_11_04.csv',
    '/kaggle/input/vevfvs/Raw X Data/inference_tweets_05_04_1.csv'
]

# Read and concatenate all CSVs
df_combined = pd.concat([pd.read_csv(fp) for fp in file_paths], ignore_index=True)

# Save to CSV
df_combined.to_csv('combined_tweets.csv', index=False)

In [ ]:
df_combined.head(10)

,text,keyword,created_at
0,This is a project that I am exceedingly happy ...,happy,2025-04-10 19:57:53+00:00
1,@RadioEuropes @Keir_Starmer @MayorofLondon Is ...,happy,2025-04-10 19:57:53+00:00
2,☘ I was just in it up in this booth off the t...,happy,2025-04-10 19:57:53+00:00
3,Happy to be of service🫡 https://t.co/bZy1IJ1BO...,happy,2025-04-10 19:57:52+00:00
4,@Eminascrypt Happy Thursday Eminas,happy,2025-04-10 19:57:51+00:00
5,@PamelaDost27628 Happy birthday!! https://t.co...,happy,2025-04-10 19:57:51+00:00
6,@Dreymwangi Haters hampendi kuniona happy😂😂😂😂😂,happy,2025-04-10 19:57:50+00:00
7,Idk wtf going on on New York Ave but I’m so ha...,happy,2025-04-10 19:57:50+00:00
8,@binibloombelge @bini_sheena The strength and ...,happy,2025-04-10 19:57:50+00:00
9,"I GOT THE PERSON I WANT, I'M HAPPY NOW BYEEEE",happy,2025-04-10 19:57:50+00:00


In [ ]:
df_combined = df_combined.drop(['keyword'], axis = 1)

In [ ]:
df_combined.to_csv('curated_dataset.csv')

Used a microsoft AI's fine tuned version of DeepSeek

In [ ]:
def detect_emotion(sentence: str, max_retries=5) -> str:
    # Initialize the client (replace with your API key)
    client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key = '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'
    )

    # Enhanced prompt with stronger directives for a single-word response
    prompt = f"""Analyze the following text and detect the emotional tone, then assign EXACTLY ONE emotion from this list: [anger, fear, joy, love, sadness].

IMPORTANT: Your response must contain ONLY a single word - one of these five emotions. No explanations, punctuation, or additional text.

Text to analyze: "{sentence}"

Primary emotion (single word only):"""

    valid_emotions = ["anger", "fear", "joy", "love", "sadness"]

    for attempt in range(max_retries):
        try:
            # Make the API call
            response = client.chat.completions.create(
                model="microsoft/mai-ds-r1:free",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=10,
                temperature=0.1,  # Even lower temperature for consistency
            )

            # Extract and clean the response
            if response.choices and response.choices[0].message.content:
                text = response.choices[0].message.content.lower().strip()

                # Direct string match first (most reliable)
                if text in valid_emotions:
                    return text

                # If not direct match, try to find any valid emotion in the response
                for emotion in valid_emotions:
                    if emotion in text:
                        return emotion

            # Wait before retrying
            time.sleep(1 + attempt)

        except Exception as e:
            print(f"API error on attempt {attempt + 1}: {str(e)}")
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)  # Exponential backoff

    # Improved fallback: Lexicon-based with better weighting
    emotion_keywords = {
        "joy": ["happy", "great", "glad", "excited", "awesome", "yay", "wonderful", "pleased",
                "delighted", "thrilled", "enjoy", "fun", "laugh", "smile", "bright", "good", "positive"],
        "sadness": ["sad", "sorry", "miss", "hurt", "pain", "disappointed", "upset", "grief",
                   "depressed", "unhappy", "crying", "tears", "regret", "lost", "alone", "bad"],
        "anger": ["angry", "hate", "mad", "damn", "fury", "outraged", "annoyed", "irritated",
                 "frustrated", "betrayed", "rage", "furious", "upset", "bitter", "resent"],
        "fear": ["scared", "fear", "afraid", "terrified", "worried", "anxious", "nervous",
                "dread", "panic", "horrified", "frightened", "concern", "stress", "terror"],
        "love": ["love", "cute", "sweet", "adore", "affection", "darling", "cherish", "care",
               "admire", "fond", "devoted", "appreciation", "passion", "warmth", "heart"]
    }

    # Count matches for each emotion
    sentence_lower = sentence.lower()
    scores = {emotion: 0 for emotion in valid_emotions}

    for emotion, keywords in emotion_keywords.items():
        for keyword in keywords:
            if keyword in sentence_lower:
                scores[emotion] += 1

    # Find emotion with highest score
    max_score = max(scores.values())

    # If we have matches, return the emotion with highest score
    if max_score > 0:
        # Get all emotions with the max score
        max_emotions = [e for e, s in scores.items() if s == max_score]
        return random.choice(max_emotions)  # Randomly choose if multiple have same score

    # Basic sentiment analysis as another fallback
    sentiment_words = {
        "positive": ["good", "nice", "great", "better", "best", "amazing", "excellent", "perfect", "beautiful"],
        "negative": ["bad", "worse", "worst", "terrible", "awful", "horrible", "poor", "wrong", "broken"]
    }

    positive_count = sum(1 for word in sentiment_words["positive"] if word in sentence_lower)
    negative_count = sum(1 for word in sentiment_words["negative"] if word in sentence_lower)

    # Map sentiment to emotions
    if positive_count > negative_count:
        return random.choice(["joy", "love"])
    elif negative_count > positive_count:
        return random.choice(["sadness", "anger", "fear"])

    # Final fallback: balanced random selection
    return random.choice(valid_emotions)

In [ ]:
df = pd.read_csv('/kaggle/input/vevfvs/Raw X Data/inference_tweets_05_04.csv')
df['emotion'] = df['text'].apply(detect_emotion)

In [ ]:
df.head()

,text,keyword,created_at,emotion
0,@OdiCrypt Please follow this guy @pithik29\n\n...,awesome,2025-04-05 19:55:36+00:00,joy
1,Awesome sunset over the city tonight! The colo...,awesome,2025-04-05 19:55:36+00:00,joy
2,Want to learn the secret to awesome game desig...,awesome,2025-04-05 19:55:36+00:00,joy
3,@imnoteru @especular_recto @SafetyAlpaca @GilA...,awesome,2025-04-05 19:55:36+00:00,joy
4,@LevisNFT This guy is awesome\n@okyzawahyu\nNe...,awesome,2025-04-05 19:55:36+00:00,anger


In [ ]:
df_combined['emotion'] = df_combined['text'].apply(detect_emotion)

In [ ]:
# Step 2: Clean the text - remove @usernames and extra whitespace
def clean_text(text):
    # Remove all @usernames
    text = re.sub(r'@\w+', '', str(text))
    # Remove extra whitespace and line breaks
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df_combined['clean_text'] = df_combined['text'].apply(clean_text)

In [ ]:
df_combined.to_csv("labeled_emotions.csv", index=False)